First we start with our classifiers:

In [9]:
import math
import random
import time
from sklearn.metrics import accuracy_score, mean_squared_error
from random import random
from sklearn import tree
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor

we imported the relevant libraries.

In [10]:
class Node():
    def __init__(self, feature=None, threshold=None, left=None, right=None, score=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.score = score
        self.value = value


class DecisionTreeClassifier():
    def __init__(self, min_samples_split=5, max_depth=3, X_idxs=[], Y_idx=[],cols=[]):
        self.root = None
        self.cols=cols
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.X_idxs = X_idxs
        self.Y_idx = Y_idx

    def build_tree(self, dataset, curr_depth=0):
        num_samples = dataset.shape[0]
        if num_samples >= self.min_samples_split and curr_depth < self.max_depth:
            num_B = dataset.loc[dataset.area_type == 1].shape[0]
            num_P = num_samples-num_B
            best_split = self.find_best_split(dataset, num_B, num_P)
            if best_split["score"] < self.get_Gini(num_B, num_P):
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
                return Node(best_split["feature"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["score"])
        leaf_value = self.leaf_value(dataset)
        return Node(value=leaf_value)

    def find_best_split(self, dataset, num_B, num_P):
        nump=dataset.to_numpy()
        best_split = {}
        max_score = float("inf")
        for feature in self.X_idxs:
            sorted_data = nump[nump[:, feature].argsort()]
            group1_B_count = 0
            group1_P_count = 0
            group2_B_count = num_B
            group2_P_count = num_P
            prev = -1
            for i in range(0,len(sorted_data)):
                if prev != -1:
                    if sorted_data[prev][feature] != sorted_data[i][feature]:
                        score = self.get_score(group1_B_count, group1_P_count, group2_B_count, group2_P_count)
                        if score <= max_score:
                            best_split["feature"] = feature
                            best_split["score"] = score
                            max_score = score
                            if prev != -1:
                                best_split["threshold"] = (sorted_data[i][feature] + sorted_data[prev][feature]) / 2
                            else:
                                best_split["threshold"] = sorted_data[i][feature]
                if sorted_data[i][self.Y_idx] == 1:
                    group1_B_count += 1
                    group2_B_count -= 1
                else:
                    group1_P_count += 1
                    group2_P_count -= 1
                prev = i
        best_split["dataset_left"], best_split["dataset_right"] = self.split(dataset, best_split["feature"],
                                                                             best_split["threshold"])
        return best_split

    def split(self, dataset, feature, threshold):
        dataset_right = dataset.loc[dataset[self.cols[feature]] >= threshold]
        dataset_left = dataset.loc[dataset[self.cols[feature]] < threshold]
        return dataset_left, dataset_right

    def get_Gini(self, ycount_1, ycount_2) -> float:
        n = ycount_1 + ycount_2
        if n == 0:
            return 0.0
        p1 = ycount_1 / n
        p2 = ycount_2 / n
        return 1.0 - (p1 ** 2 + p2 ** 2)

    def get_score(self, group1_B_count, group1_P_count, group2_B_count, group2_P_count) -> float:
        gini1 = self.get_Gini(group1_B_count, group1_P_count)
        gini2 = self.get_Gini(group2_B_count, group2_P_count)
        group1_count = group1_B_count + group1_P_count
        group2_count = group2_B_count + group2_P_count
        n = group1_count + group2_count
        gini = gini1 * (group1_count / n) + gini2 * (group2_count / n)
        return gini

    def leaf_value(self, dataset):
        values = list(dataset.iloc[:,self.Y_idx].values)
        return max(values, key=values.count)

    def predict(self, dataset):
        dataset=dataset.to_numpy()
        length=len(dataset)
        y = [self.predict_sample(index, self.root, dataset) for index in range(0, length)]
        return y

    def predict_sample(self, index, tree, dataset):
        if tree.value != None:
            return tree.value
        feature = dataset[index][tree.feature]
        if feature < tree.threshold:
            return self.predict_sample(index, tree.left, dataset)
        else:
            return self.predict_sample(index, tree.right, dataset)

above is the class of the Decision Tree, we start by defining a new class "Node" which is one node in the DT. after we defined the "DecisionTreeClassifier" class which is going to be our tree. then we defined a method that's responsible of building the tree "build_tree". after that there's a method that's responsible of finding the best split in our dataset using the B or P values "find_best_split", in it the loop goes over our dataset after it being sorted and appended into a numpy array, and checks the score for the split and checks if the score we got is better than the current score, we change it else we don't. the "split" method is used to actually split the dataset. then we calculate the gini using the "get_gini" method the "get_score" method is used to determine the gini of the current split.

at the end is our predict methods.

now we start with the Adaboost classifier class:

In [11]:
class Stump:
    def __init__(self, buildset=None, X_idxs=[], Y_idx=[],cols=[], weight=None,original_nump=None, tree=None):
        self.original_nump=original_nump
        self.weight=weight
        self.X_idxs = X_idxs
        self.Y_idx = Y_idx
        self.cols=cols
        self.tree = DecisionTreeClassifier( max_depth=1, X_idxs=X_idxs, Y_idx=Y_idx,cols=self.cols)
        self.tree.root = self.tree.build_tree(buildset)
        self.buildset = buildset
        err = self.calc_err()
        self.weight = 1 / 2 * math.log((1 - err) / err)

    def calc_err(self):
        errors = 0
        for index in range(0,len(self.original_nump)):
            pred = self.tree.predict_sample(index, self.tree.root, self.original_nump)
            if pred != self.original_nump[index][Y_index]:
                errors += self.weight[index]
        return errors


class AdaBoost:
    def __init__(self, stumps_num=None, stumps=[], X_idxs=[], Y_idx=[], cols=[]):
        self.cols=cols
        self.stumps = stumps
        self.stumps_num=stumps_num
        self.X_idxs = X_idxs
        self.Y_idx = Y_idx

    def buildAdaBoost(self, dataset,weight=[]):
        length = dataset.shape[0]
        w = 1 / length
        if(len(weight)==0):
            weight = np.full(length, w)
        nump = dataset.to_numpy()
        while(len(self.stumps)<self.stumps_num):
            wc = np.zeros(length)
            st = Stump(dataset, self.X_idxs, self.Y_idx,cols=self.cols,weight=weight,original_nump=nump)
            self.stumps.append(st)
            for index in range(0,length):
                pred = st.tree.predict_sample(index, st.tree.root, nump)
                if pred != nump[index][self.Y_idx]:
                    weight[index] *= (math.e ** (st.weight))
                else:
                    weight[index] *= (math.e ** (-st.weight))
            sum_w = sum(weight)
            weight /= sum_w
            sum_w = 0
            for index in range(0, length):
                sum_w += weight[index]
                wc[index] = sum_w
            dataset = self.make_new_dataset(nump, wc)
        return weight
    def make_new_dataset(self, nump, wc):
        sorted_rand = np.zeros(nump.shape[0])
        for i in range(0, nump.shape[0]):
            x = random()
            sorted_rand[i] = x
        sorted_rand.sort()
        new_data = np.zeros(nump.shape[0])
        i,j=0,0
        while (j < nump.shape[0]):
            if (wc[i] > sorted_rand[j]):
                new_data[j] = i
                j += 1
            else:
                i += 1
        new_data = np.int_(new_data)
        new_data_np = nump.copy()
        i = 0
        for index in new_data:
            for j in range(0, nump.shape[1]):
                new_data_np[i][j] = nump[index][j]
            i += 1
        return pd.DataFrame(new_data_np, columns=col_names)
    def predict(self, X):
        y = [self.predict_sample(index, X) for index in range(0, len(X))]
        return y

    def predict_sample(self, index, dataset):
        weight_b = 0
        weight_p = 0
        dataset=dataset.to_numpy()
        for st in self.stumps:
            res = st.tree.predict_sample(index, st.tree.root, dataset)
            if res == 1:
                weight_b += st.weight
            if res == 0:
                weight_p += st.weight
        if weight_b > weight_p:
            return 1
        else:
            return 0

we start by defining a new class "Stump" that replaces the "Node" in the decision tree, since in Adaboost each node is a stump which is a one-level-decision tree.

after that we have the "calc_err" that's used to calculate the errors we get in each run.

then we defined the adaboost class "AdaBoost". and in the end "Make_new_dataset" is responsible of adjusting our dataset according to the errors we got in the last stump.

at the end is the predict of the Adaboost class

In [12]:
#implementing 2 functions to calculate the sensitivity and specifity for the bonus section:
def sensitivity(Y,Y_pred,positive,negative):
    TP = 0
    FN = 0
    for y,y_pred in zip(Y,Y_pred):
        if(y==y_pred and y==positive):
             TP+=1
        if (y != y_pred and y_pred == negative):
             FN += 1
    return TP/(FN+TP)
def specificity(Y,Y_pred,positive,negative):
    TN = 0
    FP = 0
    for y,y_pred in zip(Y,Y_pred):
        if(y==y_pred and y==negative):
             TN+=1
        if (y != y_pred and y_pred == positive):
             FP += 1
    return TN/(FP+TN)
#################################################

define X and y and converting the data into numbers and spliting the data into training, validation, testing data

In [13]:
Y_index = 0
X_indexes = [i for i in range(1,8)]
col_names = ['area_type', 'availability', 'bedrooms', 'total_sqft', 'bath', 'balcony', 'ranked', 'price in rupees']


data = pd.read_csv("data.csv", skiprows=1, header=None, names=col_names)
map = {'B': 1, 'P': 0}
data.area_type = [map[item] for item in data.area_type]
training_data = data.iloc[:8041, :]
train_X=training_data.iloc[:,1:]
train_y=training_data.iloc[:,0]
validation_data = data.iloc[8041:10051, :]
valid_X=validation_data.iloc[:,1:]
valid_y=validation_data.iloc[:,0]
testing_data = data.iloc[10051:, :]
test_X=testing_data.iloc[:,1:]
test_y=testing_data.iloc[:,0]

now we build 10 decision trees using the hyper parameter "max-depth", then using validation data to determine which depth is best, we are checking max-depths 2-12:

In [14]:

start = time.time()
max_acc=0
best_classifier=None
for i in range(2,13):
    classifier = DecisionTreeClassifier(max_depth=i, X_idxs=X_indexes, Y_idx=Y_index,
                                        cols=col_names)
    classifier.root = classifier.build_tree(training_data)
    Y_pred = classifier.predict(validation_data)
    acc=accuracy_score(valid_y, Y_pred)
    print("accuracy of "+str(i)+" max depth is "+str(acc))
    if(acc>max_acc):
        max_acc=acc
        best_classifier=classifier
        best_sts_num=i
end = time.time()
print("best validation accuracy is using "+str(best_sts_num)+" max depth with accuracy of "+ str(max_acc))
print("Time it took for the tree to get built:"+str(end-start)+" seconds")
print("****************************************************************************************************")

accuracy of 2 max depth is 0.8950248756218906
accuracy of 3 max depth is 0.8955223880597015
accuracy of 4 max depth is 0.9014925373134328
accuracy of 5 max depth is 0.9039800995024876
accuracy of 6 max depth is 0.9099502487562189
accuracy of 7 max depth is 0.9109452736318407
accuracy of 8 max depth is 0.908955223880597
accuracy of 9 max depth is 0.9114427860696518
accuracy of 10 max depth is 0.9114427860696518
accuracy of 11 max depth is 0.9109452736318407
accuracy of 12 max depth is 0.908955223880597
best validation accuracy is using 9 max depth with accuracy of 0.9114427860696518
Time it took for the tree to get built:10.638726711273193 seconds
****************************************************************************************************


in our code, we used the "max_depth" as our hyper parameter in decision tree classifier. from the prints we can see that the program found that at the max_depth of 9 we have the best accuracy of 0.9114427860696518.

testing our decision tree best classifier with the test data:

In [15]:
Y_pred = best_classifier.predict(testing_data)
print("testing accuracy with the best classifier")
print(accuracy_score(test_y, Y_pred))

testing accuracy with the best classifier
0.910031847133758


In [16]:
print("Sensitivity is: " +str(sensitivity(test_y,Y_pred,positive=1,negative=0))+
      " and the Specificity is: "+str(specificity(test_y,Y_pred,positive=1,negative=0)))

Sensitivity is: 0.9505087881591119 and the Specificity is: 0.66


our model is much more accurate when predicting B type area, since our sensitivity is very high. yet it seems that the model is not as good as predicting P type area. A test with low specificity can be thought of as being too eager to find a positive result, even when it is not present, and may give a high number of false positives, which in our case - is classifying P as B. we're speculating that our dataset has a lot of rows that have "Area_type = B" in comparession to "Area_type = P" which is causing the classifier to be much more sensitive when identifying a "B" - resulting in high sensitivity when we set B as our positive value and a high specificity value when setting B as our negative value. our solution would be adding more data to our dataset that contain "Area_type = P" ,this will help the algorithm to learn more on P values.


implementing the <b>SKLearn</b> code to decision tree classifier:</br>
<font color='red'>NOTE: We didn't use gridsreach, we felt more comfortable using our own loop, one of the reasons was because we couldn't find online a way to use predefined validation data into gridsearch</font>

In [17]:

max_acc=0
best_classifier=None
for i in range(2,13):
    classifier = tree.DecisionTreeClassifier(min_samples_split=5,max_depth=i,random_state=10)
    classifier = classifier.fit(train_X,train_y)
    Y_pred = classifier.predict(valid_X)
    acc=accuracy_score(valid_y, Y_pred)
    print("accuracy of "+str(i)+" max depth is "+str(acc))
    if(acc>max_acc):
        max_acc=acc
        best_classifier=classifier
        best_sts_num=i
print("Decision tree classifier best validation accuracy is using "+str(best_sts_num)+" max depth with accuracy of "+ str(max_acc))
Y_pred = best_classifier.predict(test_X)
print("testing accuracy with the best classifier is:"+ str(accuracy_score(test_y, Y_pred)))

accuracy of 2 max depth is 0.8950248756218906
accuracy of 3 max depth is 0.8955223880597015
accuracy of 4 max depth is 0.9009950248756219
accuracy of 5 max depth is 0.9039800995024876
accuracy of 6 max depth is 0.9109452736318407
accuracy of 7 max depth is 0.9109452736318407
accuracy of 8 max depth is 0.9099502487562189
accuracy of 9 max depth is 0.9074626865671642
accuracy of 10 max depth is 0.9054726368159204
accuracy of 11 max depth is 0.9084577114427861
accuracy of 12 max depth is 0.9064676616915422
Decision tree classifier best validation accuracy is using 6 max depth with accuracy of 0.9109452736318407
testing accuracy with the best classifier is:0.910828025477707


We got similar results when testing our model's accuracy vs the sklearn model's accuracy when it came to the validation data, since we can see that testing the validation using our model resulted in an accuracy of 0.9114427860696518 meanwhile using sklearn we got 0.9109452736318407. meanwhile when we tested our accuracy using the "testing" data, we got a score of 0.910031847133758, in the sklearn model 0.910828025477707.

In [18]:
start = time.time()
from sklearn.metrics import accuracy_score
max_acc=0
best_classifier=None
stumps=[]
weight=[]
for i in range(10,110,10):
    classifier = AdaBoost(i, stumps, X_idxs=X_indexes, Y_idx=Y_index,cols=col_names)
    weight=classifier.buildAdaBoost(training_data,weight=weight)
    stumps=classifier.stumps
    Y_pred = classifier.predict(validation_data)
    acc=accuracy_score(valid_y, Y_pred)
    print("accuracy of "+str(i)+" stumps is "+str(acc))
    if(acc>max_acc):
        max_acc=acc
        best_classifier=classifier
        best_sts_num=i
end = time.time()
print("best validation accuracy is using "+str(best_sts_num)+" stumps with accuracy of "+ str(max_acc))
print("Time it took for the tree to get built:"+str(end-start)+" seconds")
print("****************************************************************************************************")

accuracy of 10 stumps is 0.8771144278606965
accuracy of 20 stumps is 0.8835820895522388
accuracy of 30 stumps is 0.8865671641791045
accuracy of 40 stumps is 0.8880597014925373
accuracy of 50 stumps is 0.8875621890547264
accuracy of 60 stumps is 0.8940298507462686
accuracy of 70 stumps is 0.8940298507462686
accuracy of 80 stumps is 0.8920398009950249
accuracy of 90 stumps is 0.8950248756218906
accuracy of 100 stumps is 0.8935323383084577
best validation accuracy is using 90 stumps with accuracy of 0.8950248756218906
Time it took for the tree to get built:122.50025343894958 seconds
****************************************************************************************************


in the Adaboost class we used number of stumps and checked the accuarcy every 10 stumps. we start off by creating 10 stumps and check the accuracy, then we create an additional 10 stumps on our previous adaboost model, and check the accuracy again, and so on. we can see from the prints the we got that the best validation accuracy is using 90 stumps with accuracy of 0.8950248756218906

In [19]:

Y_pred = best_classifier.predict(testing_data)
print("testing accuracy with the best classifier")
print(accuracy_score(test_y, Y_pred))

testing accuracy with the best classifier
0.8929140127388535


In [20]:
print("Sensitivity is: " +str(sensitivity(test_y,Y_pred,positive=1,negative=0))+
      " and the Specificity is: "+str(specificity(test_y,Y_pred,positive=1,negative=0)))

Sensitivity is: 0.9592969472710453 and the Specificity is: 0.4828571428571429


in comparison to the decision tree, the sensitivity is higher and the specificity is lower.

In [21]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor
max_acc=0
best_classifier=None
for i in range(10,110,10):
    classifier = AdaBoostClassifier(n_estimators=i, random_state=10)
    classifier = classifier.fit(train_X,train_y)
    Y_pred = classifier.predict(valid_X)
    acc=accuracy_score(valid_y, Y_pred)
    if(acc>max_acc):
        max_acc=acc
        best_classifier=classifier
        best_sts_num=i
print("Adaboost classifier best validation accuracy is using "+str(best_sts_num)+" max depth with accuracy of "+ str(max_acc))
Y_pred = best_classifier.predict(test_X)
print("testing accuracy with the best classifier is:"+ str(accuracy_score(test_y, Y_pred)))

Adaboost classifier best validation accuracy is using 70 max depth with accuracy of 0.9099502487562189
testing accuracy with the best classifier is:0.902468152866242


we also got similar results in Adaboost - sklearn's model being slightly better than ours.

now for our regressor:

In [22]:
class Node():
    def __init__(self, feature=None, threshold=None, left=None, right=None, score=None, value=None,samples=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.score = score
        self.value = value
        self.samples=samples

class DecisionTreeRegressor():
    def __init__(self, min_samples_split=5, max_depth=2, X_idxs=[], Y_index=[], cols=[]):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.X_idxs = X_idxs
        self.Y_idx = Y_index
        self.cols = cols
    def build_tree(self, dataset, curr_depth=0):
        num_samples = dataset.shape[0]
        self.samples=num_samples
        if num_samples >= self.min_samples_split and curr_depth < self.max_depth:
            best_split = self.find_best_split(dataset)
            if best_split["score"] < self.SSR(dataset[self.cols[self.Y_idx]]):
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)
                return Node(best_split["feature"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["score"])
        leaf_value = self.leaf_value(dataset)
        return Node(value=leaf_value,samples=len(dataset))

    def find_best_split(self, dataset):
        nump = dataset.to_numpy()
        best_split = {}
        best_split["score"] = float("inf")
        for feature in self.X_idxs:
            sorted_data = nump[nump[:, feature].argsort()]
            ssr_g2 = self.SSR(dataset[self.cols[self.Y_idx]].values)
            count_g2 = len(dataset)
            mean_g2 = dataset[self.cols[self.Y_idx]].mean()
            mean_g1, ssr_g1, count_g1, prev = 0, 0, 0, -1
            for i in range(0, len(sorted_data)):
                if prev != -1:
                    threshold = (sorted_data[i][feature] + sorted_data[prev][feature]) / 2
                else:
                    threshold = sorted_data[i][feature]
                score = ssr_g1 + ssr_g2
                if prev != -1:
                    if sorted_data[prev][feature] != sorted_data[i][feature]:
                        if score < best_split["score"]+3:#ignore the +3, it solves math accuarcy problem
                            best_split["feature"] = feature
                            best_split["score"] = score
                            best_split["threshold"] = threshold

                if (prev == -1) & (score < best_split["score"]):
                    best_split["feature"] = feature
                    best_split["score"] = score
                    best_split["threshold"] = threshold

                count_g1, mean_g1, ssr_g1 = self.add_g1(count_g1, mean_g1, ssr_g1, sorted_data[i][self.Y_idx])
                count_g2, mean_g2, ssr_g2 = self.remove_g2(count_g2, mean_g2, ssr_g2, sorted_data[i][self.Y_idx])
                prev = i

        best_split["dataset_left"], best_split["dataset_right"] = self.split(dataset, best_split["feature"],
                                                                             best_split["threshold"])
        return best_split

    def SSR(self, column):
        if (len(column) == 0):
            return 0
        avg = sum(column) / len(column)
        ssr = 0
        for c in column:
            ssr += (c - avg) ** 2
        return ssr

    def remove_g2(self, count, mean, ssr, x):
        count -= 1
        ssr -= (x - mean) ** 2
        old_me = mean
        if(count==0):
            return 0,0,0
        mean = (mean * (count + 1) - x) / (count )
        ssr -= (old_me - mean) ** 2 * count
        return count, mean, ssr

    def add_g1(self, count, mean, ssr, x):
        count += 1
        if (count == 0):
            return 0, 0, 0
        old_me = mean
        mean = (mean * (count - 1) + x) / (count )
        ssr += (old_me - mean) ** 2 * (count - 1)
        ssr += (x - mean) ** 2
        return count, mean, ssr

    def split(self, dataset, feature, threshold):
        dataset_right = dataset.loc[dataset[self.cols[feature]] >= threshold]
        dataset_left = dataset.loc[dataset[self.cols[feature]] < threshold]
        return dataset_left, dataset_right

    def leaf_value(self, dataset):
        return dataset[self.cols[self.Y_idx]].values.mean()

    def predict(self, dataset):
        dataset = dataset.to_numpy()
        length = len(dataset)
        y = [self.predict_sample(index, self.root, dataset) for index in range(0, length)]
        return y

    def predict_sample(self, index, tree, dataset):
        if tree.value != None: return tree.value
        feature_val = dataset[index][tree.feature]
        if feature_val < tree.threshold:
            return self.predict_sample(index, tree.left, dataset)
        else:
            return self.predict_sample(index, tree.right, dataset)

this is the class of the regression in the decision tree. first we start by defining a node in the tree "Node".
then we defined the class of the decision tree regressor and we build it. then we start with the "find_best_split" function, that takes the dataset and transforms it into a numpy array, and then we defined an additional array "threshold" that saves the current split. this helped us lower the run time and improve the effeciency of our code.
then we have methods that calculate the SSR and the split. at the end we have our predict functions.
for our regression we used max_depth as a hyper parameter.

In [23]:
training_data = data.iloc[:8041, :]
train_X=training_data.iloc[:,:-1]
train_y=training_data.iloc[:,-1]
validation_data = data.iloc[8041:10051, :]
valid_X=validation_data.iloc[:,:-1]
valid_y=validation_data.iloc[:,-1]
testing_data = data.iloc[10051:, :]
test_X=testing_data.iloc[:,:-1]
test_y=testing_data.iloc[:,-1]

In [24]:
X_indexes = [i for i in range(0, 7)]
Y_index = 7

start = time.time()
best_mse = float("inf")
best_regressor = None

for i in range(2, 10):
    regressor = DecisionTreeRegressor(max_depth=i, X_idxs=X_indexes, Y_index=Y_index,cols=col_names)
    regressor.root = regressor.build_tree(training_data)
    Y_pred = regressor.predict(validation_data)
    mse = mean_squared_error(valid_y, Y_pred)
    print("MSE of " + str(i) + " trees is " + str(mse))
    if mse < best_mse:
        best_mse = mse
        best_regressor = regressor
        best_sts_num = i
print("best validation MSE is using " + str(best_sts_num) +" max depth with MSE of " + str(best_mse))
Y_pred = best_regressor.predict(testing_data)
end = time.time()
print("testing MSE with the best Regressor is:")
print(mean_squared_error(test_y, Y_pred))
print("Time it took for the tree to get built:"+str(end-start)+" seconds")
print("****************************************************************************************************")

MSE of 2 trees is 101291031923336.7
MSE of 3 trees is 58287908667727.945
MSE of 4 trees is 55269670801677.19
MSE of 5 trees is 47700391589772.61
MSE of 6 trees is 44769613565845.35
MSE of 7 trees is 42703392660110.44
MSE of 8 trees is 46852250704404.92
MSE of 9 trees is 53978641156641.555
best validation MSE is using 7 max depth with MSE of 42703392660110.44
testing MSE with the best Regressor is:
59137576315808.64
Time it took for the tree to get built:20.14856195449829 seconds
****************************************************************************************************


Sklearn:

In [25]:

best_mse = float("inf")
best_regressor = None
for i in range(2, 10):
    regressor = tree.DecisionTreeRegressor(max_depth=i,random_state=10)
    regressor.fit(train_X, train_y)
    Y_pred = regressor.predict(valid_X)
    mse = mean_squared_error(valid_y, Y_pred)
    print(mse)
    if mse < best_mse:
        best_mse = mse
        best_regressor = regressor
        best_sts_num = i
print("Decision tree regressor best validation MSE is using " + str(best_sts_num) +" max depth with MSE of " + str(best_mse))
Y_pred = best_regressor.predict(test_X)
print("testing MSE with the best Regressor is:"+ str(mean_squared_error(test_y, Y_pred)))

101291031923336.7
58287908667727.945
55269670801677.19
47892602488404.46
49172922824029.09
47175647313066.36
45434652985715.12
61049482757949.61
Decision tree regressor best validation MSE is using 8 max depth with MSE of 45434652985715.12
testing MSE with the best Regressor is:65250957700329.016


we can see that we got slightly better performance in our model than in sklearn's.

our gradientBoost class:

In [26]:
class GradientBoost:
    def __init__(self, trees=[], trees_num=100, X_idxs=[], Y_idx=[], cols=[], learning_rate=0.1):
        self.learning_rate = learning_rate
        self.trees = trees
        self.trees_num = trees_num
        self.X_idxs = X_idxs
        self.Y_idx = Y_idx
        self.cols = cols

    def buildGradBoost(self, dataset, new_pred=[]):
        length = len(dataset)
        if (len(new_pred) == 0):
            new_pred = np.full(length, dataset[self.cols[self.Y_idx]].values.mean())
        y = dataset[self.cols[self.Y_idx]].to_numpy()
        while len(self.trees) < self.trees_num:
            residuals = y - new_pred
            dataset = dataset.iloc[:, :8]
            dataset.insert(8, "Residual", residuals.tolist(), True)
            self.cols.append("Residual")
            regressor = DecisionTreeRegressor(min_samples_split=3, max_depth=3, X_idxs=self.X_idxs, Y_index=8,
                                              cols=self.cols)
            regressor.root = regressor.build_tree(dataset)
            self.trees.append(regressor)
            preds = np.array(regressor.predict(dataset))
            new_pred += preds * self.learning_rate
        return new_pred

    def predict(self, dataset):
        mean = dataset[self.cols[self.Y_idx]].values.mean()
        nump = dataset.to_numpy()
        fin_pred = np.full(len(dataset), mean)
        for tree in self.trees:
            y = np.array([tree.predict_sample(index, tree.root, nump) for index in range(0, len(dataset))])
            fin_pred += self.learning_rate * y
        return fin_pred


In [27]:
from sklearn.metrics import mean_squared_error

best_mse = float("inf")
trees = []
best_regressor = None
new_pred = []
start = time.time()
for i in range(10, 151, 10):
    regressor = GradientBoost(trees=trees, trees_num=i, X_idxs=X_indexes, Y_idx=Y_index, cols=col_names)
    new_pred = regressor.buildGradBoost(training_data, new_pred)
    trees = regressor.trees
    Y_pred = regressor.predict(validation_data)
    mse = mean_squared_error(valid_y, Y_pred)
    print("mse accuracy = "+ str(mse)+ " with "+str(i)+ " trees")
    if mse < best_mse:
        best_mse = mse
        best_regressor = regressor
        best_sts_num = i
Y_pred = best_regressor.predict(testing_data)
print("GradBoost Regressor best validation MSE is using "+str(best_sts_num)+" trees with MSE of "+ str(best_mse))
end = time.time()
print("testing MSE")
print(mean_squared_error(test_y, Y_pred))
print("Time it took for the tree to get built:"+str(end-start)+" seconds")
print("****************************************************************************************************")

mse accuracy = 72474113053781.94 with 10 trees
mse accuracy = 48716032396703.43 with 20 trees
mse accuracy = 42975727677916.24 with 30 trees
mse accuracy = 41065960698005.49 with 40 trees
mse accuracy = 40754297934529.66 with 50 trees
mse accuracy = 40416925629373.23 with 60 trees
mse accuracy = 40167993278796.59 with 70 trees
mse accuracy = 39654654522216.48 with 80 trees
mse accuracy = 39385426067793.516 with 90 trees
mse accuracy = 39182041250956.55 with 100 trees
mse accuracy = 38776751102046.016 with 110 trees
mse accuracy = 38610569617295.25 with 120 trees
mse accuracy = 39171946498333.07 with 130 trees
mse accuracy = 39316331139571.945 with 140 trees
mse accuracy = 39124280394759.36 with 150 trees
GradBoost Regressor best validation MSE is using 120 trees with MSE of 38610569617295.25
testing MSE
38658262368599.2
Time it took for the tree to get built:195.78203749656677 seconds
****************************************************************************************************


In [28]:
best_mse = float("inf")
best_regressor = None
for i in range(10, 200,10):
    regressor = GradientBoostingRegressor(random_state=0,n_estimators=i,max_depth=3)
    regressor.fit(train_X, train_y)
    Y_pred = regressor.predict(valid_X)
    mse = mean_squared_error(valid_y, Y_pred)
    print(mse)
    if mse < best_mse:
        best_mse = mse
        best_regressor = regressor
        best_sts_num = i
print("Gradient boost regressor best validation MSE is using " + str(best_sts_num) +" max depth with MSE of " + str(best_mse))
Y_pred = best_regressor.predict(test_X)
print("testing MSE with the best Regressor is:"+ str(mean_squared_error(test_y, Y_pred)))

72514475577369.55
48684817969872.03
43041820051615.73
41092132130596.6
40767816496028.82
40365147154098.22
40585432545532.96
40078176820271.16
39590052219781.12
39001922201097.36
39352090659379.87
39689680509671.59
40008057300331.195
40782977851180.51
40795568024874.42
41233574315663.64
41394662572102.8
41449512115234.99
41601056425290.234
Gradient boost regressor best validation MSE is using 100 max depth with MSE of 39001922201097.36
testing MSE with the best Regressor is:39446465635117.07


our code ran slower than the sklearn's code, but we got slightly better accuracy than the sklearn code. 

<b>Performance</b>
We tried our best to cut down time complexity, we started of with really slow model and worked our way up,
we used numpy arrays instead of dataframes which really helped the runtime go better, then we used the sort method for spliting which really helped as well, then we find a way to calculate SSR and gini with O(1) , NOTE : we didn't use any help for calculating SSR with O(1), we found our original way
